# CPSC 325 Vertex AI Tutorial 


Vertex AI is an ML platform on the Google Cloud. As such it has easy integration with other google services such as BigQuery and Google Cloud Storage. This tutorial is going to specifically focus on using Vertex AI in notebooks.

There are two main types of notebooks: Managed Notebooks and User-managed notebooks. 
* Managed notebooks instances are Google-managed environments with integrations and features that help you set up and work in an end-to-end notebook-based production environment.
* User-managed notebooks are Deep Learning VM Images instances that are heavily customizable and are therefore ideal for users who need a lot of control over their environment. 

Both options are JupyterLab based and have support for both TensorFlow and PyTorch as well as GPU accelerators and the ability to sync with a Github repo. Main differences between the two are that managed notebooks are a bit easier to set up and have the ability to perform some more workflow-oriented tasks without leaving the JupyterLab interface as well as they have some options for automated notebook runs as well as automated shutdown for idle-instances. User-managed notebooks on the other hand are a lot more customizable and have better controls for networking and security needs. Also from cost perspective, it seems that User-managed notebooks are a bit cheaper. 


## Creating a Vertex AI Notebook

* Make sure that you have access to the Google Cloud Console. 
* Search for Vertex AI 
* Click on Workbench and create and new notebook in User-managed Notebooks
* Create a Notebook with your desired options

## Writing Training Code

For this demo we are going to write some simple training code to train a neural net on the MNIST dataset. The training code can be seen below.


In [1]:
import tensorflow as tf
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.datasets import mnist

# importing data
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

X_train_flat = np.array([entry.flatten() for entry in X_train])
X_test_flat = np.array([entry.flatten() for entry in X_test])

scaler = MinMaxScaler()
X_train_flat = scaler.fit_transform(X_train_flat)
X_test_flat = scaler.fit_transform(X_test_flat)

X_train_flat_train, X_val,y_train_train, y_val = train_test_split(X_train_flat,y_train,test_size=0.1)

In [2]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Reshape(target_shape=(28*28,),input_shape=(28*28,)),
    tf.keras.layers.Dense(units = 800,activation='relu'),
    tf.keras.layers.Dense(units = 800,activation='relu'),
    tf.keras.layers.Dense(units = 800,activation='relu'),
    tf.keras.layers.Dense(units = 800,activation='relu'),
    tf.keras.layers.Dense(units=10,activation='softmax')
])

model.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
history = model.fit(X_train_flat_train,y_train_train,batch_size=50,epochs=3,validation_data=(X_val,y_val))

Epoch 1/3
1080/1080 [==============================] - 32s 29ms/step - loss: 0.2243 - accuracy: 0.9334 - val_loss: 0.1507 - val_accuracy: 0.9568
Epoch 2/3
1080/1080 [==============================] - 29s 26ms/step - loss: 0.1029 - accuracy: 0.9709 - val_loss: 0.1382 - val_accuracy: 0.9622
Epoch 3/3
1080/1080 [==============================] - 33s 31ms/step - loss: 0.0794 - accuracy: 0.9774 - val_loss: 0.1298 - val_accuracy: 0.9665


## Containerizing training code
* `PROJECT_ID='your-cloud-project'`
    * To get your project id run `gcloud config list --format 'value(core.project)'`
* Convert Training notebook to .py file. 
    * `jupyter nbconvert test.ipynb --to python`
* Make a demo directory along with a trainer directory inside of it. 
* Move your `test.py` file to the trainer directory

## Creating the Dockerfile 
* In the demo directory run `touch Dockerfile` in the terminal. 
* Copy the following code into the Dockerfile. 
    ` FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-8

        WORKDIR /

        # Copies the trainer code to the docker image.
        COPY trainer /trainer

        # Sets up the entry point to invoke the trainer.
        ENTRYPOINT ["python", "-m", "trainer.test"]`
* Create a repo in Artifact Registry
    `REPO_NAME='class-demo'`

    `gcloud artifacts repositories create $REPO_NAME --repository-format=docker \`
    `--location=us-west1 --description="Docker repository" `
    * Can also make it manually in the Google Cloud Console
* Denine a variabel with the URI of the container image
    `IMAGE_URI=us-west1-docker.pkg.dev/$PROJECT_ID/$REPO_NAME`
* Configure docker 
    `gcloud auth configure-docker \`
    `us-west-docker.pkg.dev`
* Build the container
    ` docker build ./ -t $IMAGE_URI`
* Push the container to Artifcat Registry. 
    `docker push $IMAGE_URI`




# Running Custom Training Job on Vertex AI
* navigate to the training section of Vertex AI
* On Training Piplines click Create. 
* In the Container settings step, select Custom Container.
* Enter in you IMAGE_URI with your project ID. 
* Set up your machine settings and click run.  
